In [1]:
import pandas as pd
import numpy as np
import PyPDF2 as pdf
import os
from tqdm import tqdm
import re
import json
import collections


# pdfminer.six
import io
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfparser import PDFParser

## 功能
- 將分好類的至股東報告書擷取段落

## 要改的
- 加一個type選擇棄，kw2改成list-->ok
- cut_paragraph 在進入kw2十自動加上k1的起始位置-->ok
- 寫一個標題分類氣-->ok
- 條件二符合的去除實施成果-->ok
- 同一條件同位置多筆資料屬同一kw1去除重複-->ok
- 小於100字標記-->ok
- para0計算-->ok


### 準備檔案

In [2]:
df = pd.read_excel('95-107頁碼簿_已編碼merged.xlsx', dtype='str')
# df = df[df['Score']=='4']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11565 entries, 0 to 11564
Data columns (total 6 columns):
co_id        11565 non-null object
ext          11565 non-null object
filename     11565 non-null object
year         11565 non-null object
startpage    11565 non-null object
endpage      11565 non-null object
dtypes: object(6)
memory usage: 542.2+ KB


In [3]:
inputroot = os.getcwd()+'/至股東報告書已擷取'
df['inputpath'] = ''
df['filename_folder'] = ''

In [4]:
count=0
colist = []
# 讀取文件夾
for a,b,c in os.walk(inputroot):
    count+=1
    if count<=1:
        continue
    d = os.path.basename(a)
    for file in c:
        try:
            row = df.index[df['filename'].str.slice(stop=20)==file[:20]].tolist()[0]
            s = df.loc[row,'startpage']
            endpage =  df.loc[row,'endpage']
            e = str(int(endpage)-1)
            fn,ext = file.split('.')
            df.loc[row,'inputpath'] = a+'/'+fn+'.'+ext
            df.loc[row,'filename_folder'] = file
        except Exception as e: # cannot find
#             print(e)
            continue
df.head()

,co_id,ext,filename,year,startpage,endpage,inputpath,filename_folder
0,1101,pdf,2007_1101_20080613F04.pdf,96,8,9,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2007_1101_20080613F04_8to8.pdf
1,1101,pdf,2008_1101_20090619F04.pdf,97,6,7,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2008_1101_20090619F04_6to6.pdf
2,1101,pdf,2009_1101_20100618F04.pdf,98,6,7,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2009_1101_20100618F04_6to6.pdf
3,1101,pdf,2010_1101_20110622F04.pdf,99,7,8,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2010_1101_20110622F04_7to7.pdf
4,1101,pdf,2011_1101_20120621F04.pdf,100,7,8,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2011_1101_20120621F04_7to7.pdf


### 分段器

In [5]:
def extract_text_by_page(pdf_path, maxpages=0, pagenos=None): # extract whole document
    with open(pdf_path, 'rb') as fh:
        text_by_page = []
        
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      pagenos=pagenos,
                                      check_extractable=True,
                                      maxpages=maxpages):
            
            resource_manager = PDFResourceManager()
            fake_file_handle = io.StringIO()
            converter = TextConverter(resource_manager, fake_file_handle)
            page_interpreter = PDFPageInterpreter(resource_manager, converter)
            page_interpreter.process_page(page)
 
            text = fake_file_handle.getvalue()
            text_by_page.append(text)
 
            # close open handles
            converter.close()
            fake_file_handle.close()
    return text_by_page

In [6]:
def sit1loc(line):
    kw1 = ['營業', '營運', '年', '經營']
    kw2 = ['結果', '成果', '報告', '概況', '狀況', '情形', '概要']
    matches = list()
    interval = 5
    # 只找後面
    for a in kw1:   
        kw_1st = re.finditer(a,line)
        for match in kw_1st: # found mutiple kws
            for b in kw2:
                if re.search(b, line[match.start():match.end()+interval]) is not None:# found 2nd kw
                    matches.append(match)
                    matches.append([i for i in re.finditer(b, line[match.start():match.end()+interval])])

    return matches # NOT FOUND

def sit2loc(line):
    kw1 = ['營業', '營運', '年', '經營']
    kw2 = ['計畫', '計劃']
    matches = list()
    interval = 10
    # 只找後面
    for a in kw1:       
        kw_1st = re.finditer(a,line)
        for match in kw_1st: # found mutiple kws
            for b in kw2:
                if re.search(b, line[match.start():match.end()+interval]) is not None:# found 2nd kw
#                     print(match, line[match.start():match.end()+interval])
                    if sit2locexclude1(line[match.start():]):
                        continue
                    else:
                        matches.append(match)
                        matches.append([i for i in re.finditer(b, line[match.start():match.end()+interval])])
    return matches

def sit2locexclude1(line):
    kw1 = ['實施']
    kw2 = ['結果', '成果']
    matches = list()
    interval = 20
    # 只找後面
    for a in kw1:   
        kw_1st = re.finditer(a,line[:interval])
        for match in kw_1st: # found mutiple kws
            for b in kw2:
                if re.search(b, line[:interval]) is not None:# found 2nd kw
                    return 1

    return 0 # NOT FOUND

def sit3loc(line):
    kw1 = ['未來']
    kw2 = ['發展', '展望', '策略', '目標', '方向']
    matches = list()
    interval_b = 5
    interval_a = 15
    # 前後
    for a in kw1:
        kw_1st = re.finditer(a,line)
        for match in kw_1st: # found mutiple kws
            for b in kw2:
                if re.search(b, line[match.start()-interval_b:match.end()+interval_a]) is not None:# found 2nd kw
                    matches.append(match)
                    matches.append([i for i in re.finditer(b, line[match.start()-interval_b:match.end()+interval_a])])
    return matches

def sit4loc(line):
    kw1 = ['外部']
    kw2 = ['環境', '影響']
    matches = list()
    interval = 20
    # 前後
    for a in kw1:
        kw_1st = re.finditer(a,line)
        for match in kw_1st: # found mutiple kws
            for b in kw2:
                if re.search(b, line[match.start()-interval:match.end()+interval]) is not None:# found 2nd kw
                    matches.append(match)
                    matches.append([i for i in re.finditer(b, line[match.start()-interval:match.end()+interval])])
    return matches


In [7]:
def sequence_speculate(paragraph):# 推測段落的編寫順序
    # 將段落集排序，list內容drop掉    
    paragraph = drop_duplicated_keyword(paragraph)
    
    
    # 如果以中位數來區分
#     print(paragraph)
#     from statistics import median
#     lmedian1 = median(paragraph['sit1'])
#     lmedian2 = median(paragraph['sit2'])
#     lmedian3 = median(paragraph['sit3'])
#     lmedian4 = median(paragraph['sit4'])
    
#     d = {'1':lmedian1,'2':lmedian2,'3':lmedian3,'4':lmedian4}
#     print(d)
#     seq = ''
#     for item in sorted(d.items(),key=lambda t:t[1], reverse=False):
#         seq+=item[0]
#     return seq
    
    # 如果以大數法則來區分
    l1 = len(paragraph['sit1'])
    l2 = len(paragraph['sit2'])
    l3 = len(paragraph['sit3'])
    l4 = len(paragraph['sit4'])    
    lmax = max(l1,l2,l3,l4)

    combinations = list()
    for i in range(lmax):
        if i<l1:
            c1 = paragraph['sit1'][i]
        if i<l2:
            c2 = paragraph['sit2'][i]
        if i<l3:
            c3 = paragraph['sit3'][i]
        if i<l4:
            c4 = paragraph['sit4'][i]
        
        d = {'1':c1,'2':c2,'3':c3,'4':c4}
        combinations.append(collections.OrderedDict(sorted(d.items(),key=lambda t:t[1])))
    
    ctypes = {}
    for combin in combinations:
#         print(combin)
        r = ''
        for k,v in combin.items():
            r += k
        if r in ctypes:
            ctypes[r]+=1
        else:
            ctypes[r]=1
    return sorted(ctypes.items(),key=lambda t:t[1], reverse=True)[0][0]






def title_discrimination(c1,c2,c3,c4):
    cmax = max(c1,c2,c3,c4)
    cmin = min(c1,c2,c3,c4)
    cavg = (c1+c2+c3+c4)/4
    
    # 重新排序，不管情況重小排到大
    d = {'1':c1, '2':c2, '3':c3, '4':c4}
    d = sorted(d.items(),key=lambda t:t[1])
    seq=''
    for item in d:
        seq +=item[0]
#     print(seq)
    #只要含有-1就退出
    if any([c1==-1, c2==-1, c3==-1, c4==-1]):
        return -1,seq
    # check full dist -->too short return 0
    if cmax-cmin<100:
        return 1,seq
    else:
        # check ---1-------2-3-4---   
        if (d[3][1] - d[1][1])<50: 
            return 2,seq
        # check ---1-2-3-------4---  
        elif (d[2][1] - d[0][1])<50: 
            return 3,seq
        # check ---1-2-----3---4---  
        elif (d[1][1] - d[0][1])<50: 
            return 4,seq
        # check ---1---2-----3-4---
        elif (d[3][1] - d[2][1])<50: 
            return 5,seq
        # check ---1----2-3----4---
        elif (d[2][1] - d[1][1])<50: 
            return 6,seq
        # else  ---1---2---3---4---
        else: 
            return 0,seq

In [8]:
def cut_paragraph_by_sequence(paragraph, sequence='1234'):
    # 將段落集排序，list內容drop掉    
    paragraph = drop_duplicated_keyword(paragraph)

    # 定義location 無序        
    maxlocation = {
        '1':len(paragraph['sit1']),
        '2':len(paragraph['sit2']),
        '3':len(paragraph['sit3']),
        '4':len(paragraph['sit4'])        
    }
    currentlocation = {'1':0,'2':0,'3':0,'4':0}
    classlocation = {
        '1':paragraph['sit1'][currentlocation['1']],
        '2':paragraph['sit2'][currentlocation['2']],
        '3':paragraph['sit3'][currentlocation['3']],
        '4':paragraph['sit4'][currentlocation['4']]
    }
#     print(classlocation)
            
    # 有序
    isTitle, seqs = title_discrimination(classlocation[sequence[0]],classlocation[sequence[1]],classlocation[sequence[2]],classlocation[sequence[3]])
    status = True
    exception = ''
    while isTitle!=0 and isTitle!=-1 and status:
#         print('maxlocation', maxlocation)
#         print('currentlocation', currentlocation)
#         print(isTitle, seqs)
#         print('before_classlocation', classlocation)
        if all([currentlocation['1']>=maxlocation['1'],
               currentlocation['2']>=maxlocation['2'],
               currentlocation['3']>=maxlocation['3'],
               currentlocation['4']>=maxlocation['4'] ]):
            exception = 'max limit reached'
            break
        if isTitle==1: #全改
            for i in range(4): # 改後面三個
                tempint = int(seqs[i])-1
                currentlocation[sequence[tempint]] +=1
                if currentlocation[sequence[tempint]]<maxlocation[sequence[tempint]]:
                    classlocation[sequence[tempint]] = paragraph['sit'+sequence[tempint]][currentlocation[sequence[tempint]]]
                else:
                    classlocation[sequence[tempint]] = -1
                    status=False
                    exception = '~~list out of range '+str(isTitle)
                    break
        elif isTitle==2:
            for i in range(4): # 改後面三個
                tempint = int(seqs[i])-1
                if i==0: continue
                else:
                    currentlocation[sequence[tempint]] +=1
                    if currentlocation[sequence[tempint]]<maxlocation[sequence[tempint]]:
                        classlocation[sequence[tempint]] = paragraph['sit'+sequence[tempint]][currentlocation[sequence[tempint]]]
                    else:
                        classlocation[sequence[tempint]] = -1
                        status=False
                        exception = '~~list out of range '+str(isTitle)
                        break

        elif isTitle==3:
            for i in range(3): # 改前面三個
                tempint = int(seqs[i])-1
                currentlocation[sequence[tempint]] +=1
                if currentlocation[sequence[tempint]]<maxlocation[sequence[tempint]]:
                    classlocation[sequence[tempint]] = paragraph['sit'+sequence[tempint]][currentlocation[sequence[tempint]]]
                else:
                    classlocation[sequence[tempint]] = -1
                    status=False
                    exception = '~~list out of range '+str(isTitle)
                    break

        elif isTitle==4:
            for i in range(2): # 改前面兩個
                tempint = int(seqs[i])-1
                currentlocation[sequence[tempint]] +=1
                if currentlocation[sequence[tempint]]<maxlocation[sequence[tempint]]:
                    classlocation[sequence[tempint]] = paragraph['sit'+sequence[tempint]][currentlocation[sequence[tempint]]]
                else:
                    classlocation[sequence[tempint]] = -1
                    status=False
                    exception = '~~list out of range '+str(isTitle)
                    break

        elif isTitle==5:
            for i in range(4): # 改後面兩個
                tempint = int(seqs[i])-1
                if i==0 or i==1: continue
                else:
                    currentlocation[sequence[tempint]] +=1
                    if currentlocation[sequence[tempint]]<maxlocation[sequence[tempint]]:
                        classlocation[sequence[tempint]] = paragraph['sit'+sequence[tempint]][currentlocation[sequence[tempint]]]
                    else:
                        classlocation[sequence[tempint]] = -1
                        status=False
                        exception = '~~list out of range '+str(isTitle)
                        break
        elif isTitle==6:
            for i in range(3): # 改中間兩個
                tempint = int(seqs[i])-1
                if i==0: continue
                else:
                    currentlocation[sequence[tempint]] +=1
                    if currentlocation[sequence[tempint]]<maxlocation[sequence[tempint]]:
                        classlocation[sequence[tempint]] = paragraph['sit'+sequence[tempint]][currentlocation[sequence[tempint]]]
                    else:
                        classlocation[sequence[tempint]] = -1
                        status=False
                        exception = '~~list out of range '+str(isTitle)
                        break
#         print('after_classlocation', classlocation)
        isTitle, seqs = title_discrimination(classlocation[sequence[0]],classlocation[sequence[1]],classlocation[sequence[2]],classlocation[sequence[3]])
#         print(isTitle, seqs,exception)
#         print('--------')

    # 檢查順序性
    # 1,2,3,4段具順序性
    # 如果兩個數其中一個-1就跳過
    while classlocation[sequence[1]]<classlocation[sequence[0]] and classlocation[sequence[1]]!=-1 and classlocation[sequence[0]] !=-1:
        currentlocation[sequence[1]] +=1
        if currentlocation[sequence[1]]<maxlocation[sequence[1]]:
            classlocation[sequence[1]] = paragraph['sit'+sequence[1]][currentlocation[sequence[1]]]
        else: break
    while classlocation[sequence[2]]<classlocation[sequence[1]] and classlocation[sequence[2]]!=-1 and classlocation[sequence[1]] !=-1:
        currentlocation[sequence[2]] +=1
        if currentlocation[sequence[2]]<maxlocation[sequence[2]]:
            classlocation[sequence[2]] = paragraph['sit'+sequence[2]][currentlocation[sequence[2]]]
        else: break
    while classlocation[sequence[3]]<classlocation[sequence[2]] and classlocation[sequence[3]]!=-1 and classlocation[sequence[2]] !=-1:
        currentlocation[sequence[3]] +=1
        if currentlocation[sequence[3]]<maxlocation[sequence[3]]:
            classlocation[sequence[3]] = paragraph['sit'+sequence[3]][currentlocation[sequence[3]]]
        else: break
    
    
    return classlocation, exception


In [9]:
def cut_paragraph(pdf_file):
    
    texts = extract_text_by_page(pdf_file)
    paragraph = dict()
    
    loc = {'1':-1, '2':-1, '3':-1, '4':-1}
    text=seq=exception=score=fseq=''
  
    for t in texts:
        text+=t
#     print(text)
    
    # 列舉全部情況
    paragraph['sit1'] = sit1loc(text)
    paragraph['sit2'] = sit2loc(text)
    paragraph['sit3'] = sit3loc(text)
    paragraph['sit4'] = sit4loc(text)
    
    # classify letter type
    classification_type, score = letter_classification(paragraph)
    
    #debug
#     print(score)
#     print(classification_type)
#     print(paragraph)
#     print(text)
#     return loc, paragraph, score, seq, fseq, text, exception

    # 預測段落順序
    if score==4: # 如果有任何一條件是完全找不到關鍵字
        seq = sequence_speculate(dump_paragraph(paragraph))
        loc, exception = cut_paragraph_by_sequence(dump_paragraph(paragraph),sequence=seq)
        fseq=''
        for item in sorted(loc.items(),key=lambda t:t[1], reverse=False):
            fseq+=item[0]
            
        # if paragraph too short, exception
        if location_dist_too_short(loc,len(text), fseq):
            exception+='~~dist too short'
    else:
        exception = '~~some situation not found'
        
    
    return loc, paragraph, score, seq, fseq, text, exception


In [10]:
def letter_classification(paragraph):
    classification = {'c1':0, 'c2':0, 'c3':0, 'c4':0}
    
    if len(paragraph['sit1'])!=0:
        classification['c1'] = 1
    if len(paragraph['sit2'])!=0:
        classification['c2'] = 1
    if len(paragraph['sit3'])!=0:
        classification['c3'] = 1
    if len(paragraph['sit4'])!=0:
        classification['c4'] = 1

    score=0
    for k,v in classification.items():
        score+=v
         
    return classification, score

def location_dist_too_short(loc, total, fseq):
    # if distance between two paragraph is too short return 1
    # else return 0
    count=0
    for index in fseq:
        if count==0:
            pre = loc[index]
            count+=1
            continue
        else:
            dist=loc[index]-pre
            if dist<100:
                return 1
            pre = loc[index]
            count+=1
    
    if total-pre<100:
        return 1
    
    return 0
        

In [11]:
def dump_paragraph(_obj):
    # 將match物件轉換為dict與list的組合
    
    if type(_obj) ==type(dict()):
        obj = _obj.copy()  # 為避免傳址問題, 但遇到dict以外物件會出現錯誤
        for k,v in obj.items():
            templist = list()
            for i in v:
                if type(i)==type(list()):
                    templist2 = list()
                    for i2 in i:
                        templist2.append({'key':i2.group(),'start':i2.start(),'end':i2.end()})
                    templist.append(templist2)
                else:
                    templist.append({'key':i.group(),'start':i.start(),'end':i.end()})
            obj[k] = templist
    else:
        obj = str(_obj)

    return obj

def drop_duplicated_keyword(paragraph):
    # 將段落集排序，list內容drop掉    
    for k,v in paragraph.items():
        templist = list()
        for items in v:
            if type(items)==type(list()):
                pass
            else:
                templist.append(items['start'])
        paragraph[k] = sorted(list(set(templist)))
        
#     print(paragraph)
    return paragraph

In [12]:
# file = df.iloc[2570-2]['inputpath']
# print(file)
#l,p,sc,se,f,t,e = cut_paragraph(file)


### main

In [13]:
df['location'] = ''
df['text'] = ''
df['paragraph'] = ''
df['exception'] = ''
df['sequence_projected']=''
df['sequence_final']=''

In [14]:
# 開始分類每個檔案
count=0
pbar = tqdm(total=len(df))
for label, row in df.iterrows():  
    count+=1
    l=p=s=f=t=e=''
    
    try:
        file = row['inputpath']
        l,p,sc,se,f,t,e = cut_paragraph(file)
#         print(l)
#         print(p)
#         print(t)
    except Exception as ex:
        l = (-1,-1,-1,-1)
        e = str(ex)
    
    # score_calculate

    
    df.loc[label, 'exception'] = str(e)
    df.loc[label,'location'] = json.dumps(l)
    df.loc[label,'text'] = t
    df.loc[label,'paragraph'] = json.dumps(dump_paragraph(p))
    df.loc[label, 'score'] = sc
    df.loc[label,'sequence_projected'] = se
    df.loc[label,'sequence_final'] = f

    pbar.update(1)
    
#     if count==30:
#         break
pbar.close()


100%|██████████| 11565/11565 [43:32<00:00,  3.00it/s]


In [15]:
# l,p,s,t,e = cut_paragraph(df.iloc[301-2]['inputpath'])

In [16]:
df.head(30)

,co_id,ext,filename,year,startpage,endpage,inputpath,filename_folder,location,text,paragraph,exception,sequence_projected,sequence_final,score
0,1101,pdf,2007_1101_20080613F04.pdf,96,8,9,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2007_1101_20080613F04_8to8.pdf,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}",- 1 -一、 董事長致股東報告書民國96年因美國受次級房貸影響，造成美國與歐元區景氣走緩...,"{""sit1"": [{""key"": ""\u71df\u904b"", ""start"": 345...",~~some situation not found,,,1.0
1,1101,pdf,2008_1101_20090619F04.pdf,97,6,7,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2008_1101_20090619F04_6to6.pdf,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}",一、 董事長致股東報告書 民國97年因受金融風暴衝擊，全球經濟陷入嚴重衰退困境...,"{""sit1"": [{""key"": ""\u71df\u904b"", ""start"": 324...",~~some situation not found,,,1.0
2,1101,pdf,2009_1101_20100618F04.pdf,98,6,7,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2009_1101_20100618F04_6to6.pdf,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}",一、 董事長致股東報告書 民國98年受金融風暴持續影響，全球經濟在各國政府各項振興措施刺激景...,"{""sit1"": [{""key"": ""\u71df\u904b"", ""start"": 352...",~~some situation not found,,,2.0
3,1101,pdf,2010_1101_20110622F04.pdf,99,7,8,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2010_1101_20110622F04_7to7.pdf,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}",一、董事長致股東報告書 民國99年上半年全球經濟在各國政府多項振興經濟之措施下，全球經濟強...,"{""sit1"": [{""key"": ""\u71df\u904b"", ""start"": 401...",~~some situation not found,,,2.0
4,1101,pdf,2011_1101_20120621F04.pdf,100,7,8,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2011_1101_20120621F04_7to7.pdf,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}",一、董事長致股東報告書 民國100年全球經濟受限於歐債危機、歐美等先進國家就業情勢改善遲緩...,"{""sit1"": [{""key"": ""\u71df\u904b"", ""start"": 408...",~~some situation not found,,,1.0
5,1101,pdf,2012_1101_20130621F04.pdf,101,7,8,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2012_1101_20130621F04_7to7.pdf,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}",一、董事長致股東報告書 民國(以下同)101年全球經濟隨著歐債問題遲未能有效解決及就業情勢...,"{""sit1"": [{""key"": ""\u71df\u904b"", ""start"": 347...",~~some situation not found,,,1.0
6,1101,pdf,2016_1101_20170628F04.pdf,105,5,7,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2016_1101_20170628F04_5to6.pdf,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}",1 一、致股東報告書 民國（以下同）105年，台泥集團雖然面臨煤價大漲等不利因素影響，...,"{""sit1"": [{""key"": ""\u71df\u904b"", ""start"": 308...",~~some situation not found,,,1.0
7,1101,pdf,2017_1101_20180622F04.pdf,106,3,4,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2017_1101_20180622F04_3to3.pdf,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}",致股東報告書曾經有人問我，身為飯店業的Inn Keeper和水泥業的Chairman， 我是...,"{""sit1"": [{""key"": ""\u71df\u904b"", ""start"": 241...",~~some situation not found,,,2.0
8,1101,pdf,2018_1101_20190612F04.pdf,107,3,5,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2018_1101_20190612F04_3to4.pdf,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}",董事長致股東報告書水泥與藍鵲15世紀初，葡萄牙王子航海家亨利(Prince Henry th...,"{""sit1"": [{""key"": ""\u71df\u904b"", ""start"": 205...",~~some situation not found,,,1.0
9,1102,pdf,2007_1102_20080617FE4.pdf,96,4,13,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2007_1102_20080617FE4_4to12.pdf,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}",- 1 - 致股東報告書Report to Shareholders 壹、致股東報告書 壹...,"{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 310...",~~some situation not found,,,2.0


In [17]:
df['exception'].value_counts()

~~some situation not found               6701
                                         3236
~~dist too short                          789
~~list out of range 5~~dist too short     354
~~list out of range 5                     186
~~list out of range 4                      75
~~list out of range 6                      72
~~list out of range 6~~dist too short      40
~~list out of range 4~~dist too short      36
~~list out of range 2~~dist too short      33
~~list out of range 3~~dist too short      20
~~list out of range 1~~dist too short      10
~~list out of range 2                       7
~~list out of range 3                       6
Name: exception, dtype: int64

#### 後處理

In [18]:
df['Classified'] = None
for label, row in df.iterrows():
    if df.loc[label,'score'] == 4:
        df.loc[label, 'Classified'] = '可完整拆分'
    elif df.loc[label,'score'] == 0:
        df.loc[label, 'Classified'] = '無法拆分'
    else:
        df.loc[label, 'Classified'] = '可部分拆分'


In [19]:
df['Classified'].value_counts()

可部分拆分    5947
可完整拆分    4864
無法拆分      754
Name: Classified, dtype: int64

In [20]:
df.to_excel('95-107頁碼簿_分段狀況.xlsx',index=False)
df.head(15)

,co_id,ext,filename,year,startpage,endpage,inputpath,filename_folder,location,text,paragraph,exception,sequence_projected,sequence_final,score,Classified
0,1101,pdf,2007_1101_20080613F04.pdf,96,8,9,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2007_1101_20080613F04_8to8.pdf,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}",- 1 -一、 董事長致股東報告書民國96年因美國受次級房貸影響，造成美國與歐元區景氣走緩...,"{""sit1"": [{""key"": ""\u71df\u904b"", ""start"": 345...",~~some situation not found,,,1.0,可部分拆分
1,1101,pdf,2008_1101_20090619F04.pdf,97,6,7,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2008_1101_20090619F04_6to6.pdf,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}",一、 董事長致股東報告書 民國97年因受金融風暴衝擊，全球經濟陷入嚴重衰退困境...,"{""sit1"": [{""key"": ""\u71df\u904b"", ""start"": 324...",~~some situation not found,,,1.0,可部分拆分
2,1101,pdf,2009_1101_20100618F04.pdf,98,6,7,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2009_1101_20100618F04_6to6.pdf,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}",一、 董事長致股東報告書 民國98年受金融風暴持續影響，全球經濟在各國政府各項振興措施刺激景...,"{""sit1"": [{""key"": ""\u71df\u904b"", ""start"": 352...",~~some situation not found,,,2.0,可部分拆分
3,1101,pdf,2010_1101_20110622F04.pdf,99,7,8,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2010_1101_20110622F04_7to7.pdf,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}",一、董事長致股東報告書 民國99年上半年全球經濟在各國政府多項振興經濟之措施下，全球經濟強...,"{""sit1"": [{""key"": ""\u71df\u904b"", ""start"": 401...",~~some situation not found,,,2.0,可部分拆分
4,1101,pdf,2011_1101_20120621F04.pdf,100,7,8,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2011_1101_20120621F04_7to7.pdf,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}",一、董事長致股東報告書 民國100年全球經濟受限於歐債危機、歐美等先進國家就業情勢改善遲緩...,"{""sit1"": [{""key"": ""\u71df\u904b"", ""start"": 408...",~~some situation not found,,,1.0,可部分拆分
5,1101,pdf,2012_1101_20130621F04.pdf,101,7,8,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2012_1101_20130621F04_7to7.pdf,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}",一、董事長致股東報告書 民國(以下同)101年全球經濟隨著歐債問題遲未能有效解決及就業情勢...,"{""sit1"": [{""key"": ""\u71df\u904b"", ""start"": 347...",~~some situation not found,,,1.0,可部分拆分
6,1101,pdf,2016_1101_20170628F04.pdf,105,5,7,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2016_1101_20170628F04_5to6.pdf,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}",1 一、致股東報告書 民國（以下同）105年，台泥集團雖然面臨煤價大漲等不利因素影響，...,"{""sit1"": [{""key"": ""\u71df\u904b"", ""start"": 308...",~~some situation not found,,,1.0,可部分拆分
7,1101,pdf,2017_1101_20180622F04.pdf,106,3,4,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2017_1101_20180622F04_3to3.pdf,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}",致股東報告書曾經有人問我，身為飯店業的Inn Keeper和水泥業的Chairman， 我是...,"{""sit1"": [{""key"": ""\u71df\u904b"", ""start"": 241...",~~some situation not found,,,2.0,可部分拆分
8,1101,pdf,2018_1101_20190612F04.pdf,107,3,5,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2018_1101_20190612F04_3to4.pdf,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}",董事長致股東報告書水泥與藍鵲15世紀初，葡萄牙王子航海家亨利(Prince Henry th...,"{""sit1"": [{""key"": ""\u71df\u904b"", ""start"": 205...",~~some situation not found,,,1.0,可部分拆分
9,1102,pdf,2007_1102_20080617FE4.pdf,96,4,13,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2007_1102_20080617FE4_4to12.pdf,"{""1"": -1, ""2"": -1, ""3"": -1, ""4"": -1}",- 1 - 致股東報告書Report to Shareholders 壹、致股東報告書 壹...,"{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 310...",~~some situation not found,,,2.0,可部分拆分


### 拆段落

In [21]:
# raise Exception

In [22]:
df1 = df.copy()
df1 = df1[df1['sequence_final']!='']
df1.head()

,co_id,ext,filename,year,startpage,endpage,inputpath,filename_folder,location,text,paragraph,exception,sequence_projected,sequence_final,score,Classified
15,1103,pdf,2007_1103_20080613F04.pdf,96,5,9,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2007_1103_20080613F04_5to8.pdf,"{""1"": 303, ""2"": 1393, ""3"": 3051, ""4"": 2397}",壹、致股東報告書 各位股東先生： 民國九十六年的國際經濟情勢不佳，包括國際油價持續飆漲、各...,"{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 321...",,1243,1243,4.0,可完整拆分
16,1103,pdf,2008_1103_20090619F04.pdf,97,5,8,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2008_1103_20090619F04_5to7.pdf,"{""1"": 258, ""2"": 1537, ""3"": 2950, ""4"": 2471}",壹、致股東報告書 各位股東先生： 去年（97年）美國次級房貸風暴所引起的「金融大海嘯」，...,"{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 276...",,1243,1243,4.0,可完整拆分
17,1103,pdf,2009_1103_20100618F04.pdf,98,5,9,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2009_1103_20100618F04_5to8.pdf,"{""1"": 117, ""2"": 1534, ""3"": 3257, ""4"": 2561}",壹、致股東報告書 各位股東先生： 遭逢史上罕見之金融大海嘯所帶來之嚴重衝擊，各國紛紛提出振...,"{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 134...",,1243,1243,4.0,可完整拆分
18,1103,pdf,2010_1103_20110609F04.pdf,99,5,10,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2010_1103_20110609F04_5to9.pdf,"{""1"": 142, ""2"": 1731, ""3"": 3304, ""4"": 2442}",3 壹、致股東報告書 各位股東先生女士： 歷經金融大海嘯後之休養生息，景氣的復甦有感可見，...,"{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 160...",,1243,1243,4.0,可完整拆分
24,1103,pdf,2016_1103_20170619F04.pdf,105,6,10,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2016_1103_20170619F04_6to9.pdf,"{""1"": 488, ""2"": 2619, ""3"": 1851, ""4"": 1840}",壹、致股東報告書 各位股東女士、先生： 105年度國內景氣受國際經濟情勢緩升影響，自...,"{""sit1"": [{""key"": ""\u71df\u696d"", ""start"": 488...",~~dist too short,1432,1432,4.0,可完整拆分


In [23]:
def extract_paragraph(loc, fulltext, seq='1234'): 

    # 四個關鍵位置有任一個是-1，有段落合寫
    skip_seq = dict()
    for k,v in loc.items():
        if v==-1:
            skip_seq[k]=True
        else:
            skip_seq[k]=False
#     print(skip_seq)
    
    # 標註段落
    seq = seq[::-1] # reverse
    paragraph_text = {'1':'','2':'','3':'','4':''}
    post = len(fulltext)
    count = 0
    for s in seq:
#         print('skip?',skip_seq[s], end='\t')
#         print('post:',post, end='\t')
#         print('current:',loc[seq[count]], end='\t')
#         print('seq:',s, end='\n')
        if not skip_seq[s]:
            paragraph_text[s] = fulltext[loc[seq[count]]:post]
            post = loc[seq[count]] 
        count+=1
    paragraph_text['0'] = fulltext[:post]
    # 計算長度
    paragraph_length = {'0':0,'1':0,'2':0,'3':0,'4':0}
    for k,v in paragraph_text.items():
        paragraph_length[k] = len(v)
    
    return paragraph_text, paragraph_length

In [24]:
# 開始分類每個檔案
count=0
pbar = tqdm(total=len(df1))
colist = []
for label, row in df1.iterrows():   
    try:
        location = json.loads(row['location'])
        text = row['text']
        sequence = row['sequence_final']
        pt, pl = extract_paragraph(location, text, sequence)
        colist.append({
            'file':row['inputpath'],
            'filename': row['filename_folder'],
            'text': row['text'],
            'location': row['location'],
            'exception':row['exception'],
            'lenpara0':pl['0'],
            'lenpara1':pl['1'],
            'lenpara2':pl['2'],
            'lenpara3':pl['3'],
            'lenpara4':pl['4'],
            'lentotal':len(row['text']),
            'para0':pt['0'],
            'para1':pt['1'],
            'para2':pt['2'],
            'para3':pt['3'],
            'para4':pt['4'],
            'sequence':sequence
        })
    except Exception as e:
        print(e)
        print(row['filename_folder'])
        
    pbar.update(1)
    
#     if count==15:
#         break
pbar.close()
dfr = pd.DataFrame(colist)
dfr.head(30)

100%|██████████| 4864/4864 [00:00<00:00, 8175.33it/s]


,exception,file,filename,lenpara0,lenpara1,lenpara2,lenpara3,lenpara4,lentotal,location,para0,para1,para2,para3,para4,sequence,text
0,,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2007_1103_20080613F04_5to8.pdf,303,1090,1004,717,654,3768,"{""1"": 303, ""2"": 1393, ""3"": 3051, ""4"": 2397}",壹、致股東報告書 各位股東先生： 民國九十六年的國際經濟情勢不佳，包括國際油價持續飆漲、各...,營運狀況作一報告： 一、九十六年度營業結果： 本公司九十六年度全年水泥及半成品之銷...,年營業計畫概要： 展望九十七年，本公司仍將繼續九十六年的工作，致力水泥本業、物流事業...,未來的發展 公司未來將持續發展三大核心產業，一為水泥本業的持續拓展，二為港口物流事業...,外部環境的變化： （一）外部環境的變化 國內在新政府上任後，推行愛台十二項建設，對提...,1243,壹、致股東報告書 各位股東先生： 民國九十六年的國際經濟情勢不佳，包括國際油價持續飆漲、各...
1,,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2008_1103_20090619F04_5to7.pdf,258,1279,934,462,479,3412,"{""1"": 258, ""2"": 1537, ""3"": 2950, ""4"": 2471}",壹、致股東報告書 各位股東先生： 去年（97年）美國次級房貸風暴所引起的「金融大海嘯」，...,營運狀況作一報告： 一、九十七年度營業結果： 本公司九十七年度之銷貨收入為新台幣（...,年營業計畫概要： 展望九十八年，由於煤炭價格的下跌，毛利率將回升，獲利也會增加；再者...,未來的發展 水泥本業的發展，在策略發展方向，將持續追求內部增長擴建，轉投資之台泥國際...,"外部環境的變化 國內4年5,000億元新台幣擴大公共建設，重點工程包括機場捷運、台中...",1243,壹、致股東報告書 各位股東先生： 去年（97年）美國次級房貸風暴所引起的「金融大海嘯」，...
2,,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2009_1103_20100618F04_5to8.pdf,117,1417,1027,893,696,4150,"{""1"": 117, ""2"": 1534, ""3"": 3257, ""4"": 2561}",壹、致股東報告書 各位股東先生： 遭逢史上罕見之金融大海嘯所帶來之嚴重衝擊，各國紛紛提出振...,營運狀況作一報告： 一、九十八年度營業結果： 本公司九十八年度全年之銷貨收入為新台幣（以下同...,"年營業計畫概要： 展望九十九年，國內延續去（98）年振興經濟的方向，今（99）年仍有6,50...",未來的發展 嘉新過去以生產為導向，然面對兩岸市場大環境的變化，水泥產業朝向大者恆大的趨勢發展...,外部環境的變化 海峽兩岸開放三通直航後，經貿往來日漸頻繁，繼兩岸金融監理合作備忘錄（MOU）...,1243,壹、致股東報告書 各位股東先生： 遭逢史上罕見之金融大海嘯所帶來之嚴重衝擊，各國紛紛提出振...
3,,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2010_1103_20110609F04_5to9.pdf,142,1589,711,529,862,3833,"{""1"": 142, ""2"": 1731, ""3"": 3304, ""4"": 2442}",3 壹、致股東報告書 各位股東先生女士： 歷經金融大海嘯後之休養生息，景氣的復甦有感可見，...,營運狀況作一報告： 一、九十九年度營業結果： 本公司九十九年度全年之銷貨收入為新台...,"年營業計畫概要： 國內今（100）年有5,000億元新台幣投入公共建設，包括蘇花改、機場捷...",未來的發展策略 嘉新在水泥本業發展策略方向上，仍秉持相同理念，繼續以持股投資的方式，與龍頭企...,外部環境的變化 兩岸經濟合作架構協議（ECFA）於今年1月1日生效，經合會（兩岸經濟合作委員...,1243,3 壹、致股東報告書 各位股東先生女士： 歷經金融大海嘯後之休養生息，景氣的復甦有感可見，...
4,~~dist too short,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2016_1103_20170619F04_6to9.pdf,488,1352,309,768,11,2928,"{""1"": 488, ""2"": 2619, ""3"": 1851, ""4"": 1840}",壹、致股東報告書 各位股東女士、先生： 105年度國內景氣受國際經濟情勢緩升影響，自...,營業狀況做一報告： 一、去年度營業結果 本公司105年度合併各子公司之營業收入為新台幣（以下...,年度營業計畫概要 本公司106年度預計水泥銷量、房產租賃、及台北港第一散雜貨中心裝-...,未來發展策略 儘管國內經濟景氣好轉，然而房地產市場尚未完全落底，盤整猶須時日，短期間...,外部環境變動影響與公司,1432,壹、致股東報告書 各位股東女士、先生： 105年度國內景氣受國際經濟情勢緩升影響，自...
5,~~list out of range 6,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2017_1103_20180621F04_6to9.pdf,550,1504,434,129,0,2617,"{""1"": 550, ""2"": 2183, ""3"": 2054, ""4"": -1}",壹、致股東報告書 各位股東女士、先生： 106年度隨著先進、新興與開發中經濟體普遍復甦...,營業狀況做一報告: 一、去年度營業結果 本公司106年度合併各子公司之營業收入為2...,年度營業計畫概要 本公司107年度預計水泥銷量、房產租賃等營運目標基本與106年度相...,未來發展策略 未來發展上，本公司繼續朝向轉型邁進，但由於升息循環即將展開，國內、外情...,,4132,壹、致股東報告書 各位股東女士、先生： 106年度隨著先進、新興與開發中經濟體普遍復甦...
6,~~list out of range 5,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2018_1103_20190621F04_6to9.pdf,956,1767,333,126,0,3182,"{""1"": 956, ""2"": 2849, ""3"": 2723, ""4"": -1}",壹、致股東報告書 各位股東女士、先生： 民國107年度台灣經濟受全球景氣持續擴張的帶...,"營業成果: 一、去年度營業結果 本公司107年度合併各子公司之營業收入為2,092...",年度營業計畫概要 本公司108年度預計水泥銷量、房產租賃等事業之營運目標，基本與10...,未來發展策略 未來本公司將著重穩固轉型後的產業營運，以及儘早建立旅宿服務與健康生活事...,,4132,壹、致股東報告書 各位股東女士、先生： 民國107年度台灣經濟受全球景氣持續擴張的帶...
7,,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2006_1104_20070622F04_5to14.pdf,1834,3394,270,361,648,6507,"{""1"": 1834, ""2"": 5228, ""3"": 5498, ""4"": 5859}",～ 1 ～ 壹、致股東報告書 一、董事長致詞 各位敬愛的股東女士先生 九十五年的國內經濟景氣...,"年度營業結果 (一)生 產 (1)水泥 本公司95年度生產水泥551,300公噸（阿蓮水泥廠...",年度營業計畫概要 (一)經營方針及預期銷售量 1.生產水泥55萬公噸、銷售水泥59萬公噸。 ...,未來公司發展策略 (一)國內 1.續精耕本業，開源節流，提升營業利益。 2.評估進行閒置資產...,外部競爭環境、法規環境及總體經營環境之影響 (一)外部競爭環境影響 1.水泥 (1)南部地區...,1234,～ 1 ～ 壹、致股東報告書 一、董事長致詞 各位敬愛的股東女士先生 九十五年的國內經濟景氣...
8,,C:\Users\Sing\Documents\年報合併\年報95年-107年按產業分類/至...,2007_1104_20080613F04_4to13.pdf,1764,3470,294,311,799,6638,"{""1"": 1764, ""2"": 5234, ""3"": 5528, ""4"": 5839}",- 1 - 壹、致股東報告書 一、董事長致詞 各位敬愛的股東女士先生 九十六年度國內經濟景氣...,"年度營業結果 (一)生 產 (1)水泥 本公司96年度生產水泥446,970公噸（阿蓮水泥廠...",年度營業計畫概要 (一)經營方針及預期銷售量 1.

In [25]:
dfr.to_excel('95-107頁碼簿_分段結果.xlsx',index=False)